In [1]:
import tensorflow as tf
from tensorflow.contrib import seq2seq
from elmo import ELMo
from data import NERData
import os

total_epoch = 5000
hidden_size = 200
vocab_size = 5000
max_length = 128
entity_class = 7

batch_size = 1    # 跟训练的区别

ner = NERData(batch_size, max_length)
elmo = ELMo(batch_size, hidden_size, vocab_size)

/home/albert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


成功加载语料data/example.train, 语料数量18404
成功加载语料data/example.test, 语料数量4558
成功加载语料data/example.dev, 语料数量4384


In [2]:
def network(X):
    w = tf.get_variable("fcn_w", [1, hidden_size, entity_class + 1])
    b = tf.get_variable("fcn_b", [entity_class + 1])
    # 这里输出维度用entity_class + 1而不是entity_class，因为输出里除了7类实体，还有一类用来表示每个句子补齐的<PAD>位
    w_tile = tf.tile(w, [batch_size, 1, 1])

    logists = tf.nn.softmax(tf.nn.xw_plus_b(X, w_tile, b), name="logists")
    return logists

In [3]:
X = tf.placeholder(shape=[batch_size, max_length], dtype=tf.int32, name="X")
length = tf.placeholder(shape=[batch_size], dtype=tf.int32, name="length")
dropout = tf.placeholder(shape=[], dtype=tf.float32, name="dropout")

elmo_vector = elmo.elmo(X, length, dropout)
logists = network(elmo_vector)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
model_dir = "./model"
saver = tf.train.Saver()
check_point = tf.train.get_checkpoint_state(model_dir)
saver.restore(sess, check_point.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from ./model/model-150000


#### 查看elmo各层的权重

In [4]:
_X, _length, _targets = ner.get_dev_data(1)
fd = {X: _X, length: _length, dropout: 1.}
weights = sess.run(tf.nn.softmax(elmo.weights), feed_dict=fd)
print("embedding层:", weights[0])
print("forward_1层:", weights[1])
print("forward_2层:", weights[2])
print("backward_1层:", weights[3])
print("backward_2层:", weights[4])

embedding层: 0.00021379402
forward_1层: 0.028424246
forward_2层: 0.50437975
backward_1层: 0.02824858
backward_2层: 0.43873364


#### 用来自不同领域的句子进行测试

In [5]:
s = "李克强来到位于江西省赣州市于都县的梓山镇潭头村看望慰问群众"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('李', 'B-PER'),
 ('克', 'I-PER'),
 ('强', 'I-PER'),
 ('来', 'O'),
 ('到', 'O'),
 ('位', 'O'),
 ('于', 'O'),
 ('江', 'B-LOC'),
 ('西', 'I-LOC'),
 ('省', 'I-LOC'),
 ('赣', 'B-LOC'),
 ('州', 'I-LOC'),
 ('市', 'I-LOC'),
 ('于', 'B-LOC'),
 ('都', 'I-LOC'),
 ('县', 'I-LOC'),
 ('的', 'O'),
 ('梓', 'B-LOC'),
 ('山', 'I-LOC'),
 ('镇', 'I-LOC'),
 ('潭', 'I-LOC'),
 ('头', 'I-LOC'),
 ('村', 'I-LOC'),
 ('看', 'O'),
 ('望', 'O'),
 ('慰', 'O'),
 ('问', 'O'),
 ('群', 'O'),
 ('众', 'O')]

In [6]:
s = "多名白宫官员对媒体表示，美国总统特朗普不准备续签即将到期的美俄《新削减战略武器条约》，而是想推动达成一项包括中国在内的新条约"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('多', 'O'),
 ('名', 'O'),
 ('白', 'B-ORG'),
 ('宫', 'I-ORG'),
 ('官', 'O'),
 ('员', 'O'),
 ('对', 'O'),
 ('媒', 'O'),
 ('体', 'O'),
 ('表', 'O'),
 ('示', 'O'),
 ('，', 'O'),
 ('美', 'B-LOC'),
 ('国', 'I-LOC'),
 ('总', 'O'),
 ('统', 'O'),
 ('特', 'B-PER'),
 ('朗', 'I-LOC'),
 ('普', 'I-PER'),
 ('不', 'O'),
 ('准', 'O'),
 ('备', 'O'),
 ('续', 'O'),
 ('签', 'O'),
 ('即', 'O'),
 ('将', 'O'),
 ('到', 'O'),
 ('期', 'O'),
 ('的', 'O'),
 ('美', 'B-LOC'),
 ('俄', 'B-LOC'),
 ('《', 'O'),
 ('新', 'O'),
 ('削', 'O'),
 ('减', 'O'),
 ('战', 'O'),
 ('略', 'O'),
 ('武', 'O'),
 ('器', 'O'),
 ('条', 'O'),
 ('约', 'O'),
 ('》', 'O'),
 ('，', 'O'),
 ('而', 'O'),
 ('是', 'O'),
 ('想', 'O'),
 ('推', 'O'),
 ('动', 'O'),
 ('达', 'O'),
 ('成', 'O'),
 ('一', 'O'),
 ('项', 'O'),
 ('包', 'O'),
 ('括', 'O'),
 ('中', 'B-LOC'),
 ('国', 'I-LOC'),
 ('在', 'O'),
 ('内', 'O'),
 ('的', 'O'),
 ('新', 'O'),
 ('条', 'O'),
 ('约', 'O')]

In [7]:
s = "小罗伯特·唐尼专门为漫威总裁凯文费奇颁发特别大奖，他在介绍凯文费奇的时候说道“我要感谢凯文，他在我的低谷期认可我“"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('小', 'B-PER'),
 ('罗', 'B-PER'),
 ('伯', 'I-PER'),
 ('特', 'I-PER'),
 ('·', 'I-PER'),
 ('唐', 'I-PER'),
 ('尼', 'I-PER'),
 ('专', 'I-PER'),
 ('门', 'O'),
 ('为', 'O'),
 ('漫', 'O'),
 ('威', 'O'),
 ('总', 'O'),
 ('裁', 'O'),
 ('凯', 'B-PER'),
 ('文', 'I-PER'),
 ('费', 'I-PER'),
 ('奇', 'I-PER'),
 ('颁', 'O'),
 ('发', 'O'),
 ('特', 'O'),
 ('别', 'O'),
 ('大', 'O'),
 ('奖', 'O'),
 ('，', 'O'),
 ('他', 'O'),
 ('在', 'O'),
 ('介', 'O'),
 ('绍', 'O'),
 ('凯', 'B-PER'),
 ('文', 'I-PER'),
 ('费', 'I-PER'),
 ('奇', 'I-PER'),
 ('的', 'O'),
 ('时', 'O'),
 ('候', 'O'),
 ('说', 'O'),
 ('道', 'O'),
 ('“', 'O'),
 ('我', 'O'),
 ('要', 'O'),
 ('感', 'O'),
 ('谢', 'O'),
 ('凯', 'B-PER'),
 ('文', 'I-PER'),
 ('，', 'O'),
 ('他', 'O'),
 ('在', 'O'),
 ('我', 'O'),
 ('的', 'O'),
 ('低', 'O'),
 ('谷', 'O'),
 ('期', 'O'),
 ('认', 'O'),
 ('可', 'O'),
 ('我', 'O'),
 ('“', 'O')]

In [8]:
s = "马苏、周冬雨、林更新、霍建华都被拍到出现在周杰伦的演唱会上，此外，林俊杰和陈奕迅等超级巨星也都作为演唱会嘉宾出现在演唱会上"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('马', 'B-LOC'),
 ('苏', 'I-LOC'),
 ('、', 'O'),
 ('周', 'B-PER'),
 ('冬', 'I-PER'),
 ('雨', 'I-PER'),
 ('、', 'O'),
 ('林', 'B-PER'),
 ('更', 'I-PER'),
 ('新', 'I-PER'),
 ('、', 'O'),
 ('霍', 'B-PER'),
 ('建', 'I-PER'),
 ('华', 'I-PER'),
 ('都', 'O'),
 ('被', 'O'),
 ('拍', 'O'),
 ('到', 'O'),
 ('出', 'O'),
 ('现', 'O'),
 ('在', 'O'),
 ('周', 'B-PER'),
 ('杰', 'I-PER'),
 ('伦', 'I-PER'),
 ('的', 'O'),
 ('演', 'O'),
 ('唱', 'O'),
 ('会', 'O'),
 ('上', 'O'),
 ('，', 'O'),
 ('此', 'O'),
 ('外', 'O'),
 ('，', 'O'),
 ('林', 'B-PER'),
 ('俊', 'I-PER'),
 ('杰', 'I-PER'),
 ('和', 'O'),
 ('陈', 'B-PER'),
 ('奕', 'I-PER'),
 ('迅', 'I-PER'),
 ('等', 'O'),
 ('超', 'O'),
 ('级', 'O'),
 ('巨', 'O'),
 ('星', 'O'),
 ('也', 'O'),
 ('都', 'O'),
 ('作', 'O'),
 ('为', 'O'),
 ('演', 'O'),
 ('唱', 'O'),
 ('会', 'O'),
 ('嘉', 'O'),
 ('宾', 'O'),
 ('出', 'O'),
 ('现', 'O'),
 ('在', 'O'),
 ('演', 'O'),
 ('唱', 'O'),
 ('会', 'O'),
 ('上', 'O')]